In [1]:
from random import shuffle
import glob, h5py, pickle, cv2, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
sub = 'twoAct'

trainpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/train_' + sub + '/*_orig.png'
testpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/test_' + sub + '/*_orig.png'
filename = 'datasets/' + sub + '_orig.hdf5'
print('File to be created: ' + filename)

File to be created: datasets/twoAct_orig.hdf5


In [2]:
train_files2 = sorted(glob.glob(trainpath))
print(len(train_files2))
train_images = []
cnt = 1
train_labels = []
for i in range(len(train_files2)):
    fname = train_files2[i].split('/')[-1]
    if '\\' in fname:
        fname = fname.split('\\')[-1]
    class_idx = [m.end() for m in re.finditer('class', fname)]
    ang_idx = [m.end() for m in re.finditer('ang', fname)]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    classes = []
    classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
    classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
    classes = np.array(classes)
    
    for i in range(len(classes)):
        if classes[i] == 13:
            classes[i] = 8
        elif classes[i] == 14:
            classes[i] = 9
    train_labels.append(classes-1)
    img = cv2.imread(train_files2[i])
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(train_files2))+ ' ' + fname)
    train_images.append(img)  
    cnt += 1
print(np.unique(train_labels))
train_labels = to_categorical(np.array(train_labels))
train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
print(train_images.shape)
print(train_labels.shape)

Loading image 499/499 pSpot_class4_ang0_iter9_pCemre_class7_ang+45_iter5_orig.png
[0 1 2 3 4 5 6 7 8]
(499, 128, 128, 3)
(499, 2, 9)


In [3]:
test_files2 = sorted(glob.glob(testpath))
print(len(test_files2))
test_images = []
cnt = 1
test_labels = []
for i in range(len(test_files2)):
    fname = test_files2[i].split('/')[-1]
    if '\\' in fname:
        fname = fname.split('\\')[-1]
    class_idx = [m.end() for m in re.finditer('class', fname)]
    ang_idx = [m.end() for m in re.finditer('ang', fname)]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    classes = []
    classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
    classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
    classes = np.array(classes)
    
    for i in range(len(classes)):
        if classes[i] == 13:
            classes[i] = 8
        elif classes[i] == 14:
            classes[i] = 9
    test_labels.append(classes-1)
    img = cv2.imread(test_files2[0])
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(test_files2))+ ' ' + fname)
    test_images.append(img)  
    cnt += 1
print(np.unique(test_labels))
test_labels = to_categorical(np.array(test_labels))
test_images = np.swapaxes(np.array(test_images), 1, 2).reshape(len(test_images), width, height, channels)/255.
print(test_images.shape)
print(test_labels.shape)

Loading image 245/245 pSpot_class4_ang0_iter8_pSpot_class3_ang-45_iter1_orig.png
[0 1 2 3 4 5 6 7 8]
(245, 128, 128, 3)
(245, 2, 9)


In [4]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", train_images.shape, np.uint8)
f.create_dataset("test_img", test_images.shape, np.uint8)  
f.create_dataset("train_labels", train_labels.shape, np.uint8)
f.create_dataset("test_labels", test_labels.shape, np.uint8)

f["train_img"][...] = train_images
f["test_img"][...] = test_images
f["train_labels"][...] = train_labels
f["test_labels"][...] = test_labels
f.close()
print(filename+' created.')

datasets/twoAct_orig.hdf5 created.
